In [1]:
from keras.models import load_model
import pandas as pd
import numpy as np
from PIL import Image,ImageOps
import CharacterSegmentation as cs
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os

In [ ]:
INPUT_IMAGE = './input/input_1.jpeg'
SEGMENTED_OUTPUT_DIR = './segmented/'
EMNIST_PATH = './data/'
MODEL_PATH = './model/alphanum_model_binary_ex_88.h5'
mapping_processed = EMNIST_PATH + 'processed-mapping.csv'

: 

## Character Segmentation

In [ ]:
img = Image.open(INPUT_IMAGE)
plt.imshow(img)

: 

In [ ]:
cs.image_segmentation(INPUT_IMAGE)

: 

In [ ]:
segmented_images = []
files = [f for r, d, f in os.walk(SEGMENTED_OUTPUT_DIR)][0]
for f in files:
    segmented_images.append(Image.open(SEGMENTED_OUTPUT_DIR + f))

: 

In [ ]:
figure(figsize=(18,18))

size = len(segmented_images)
for i in range(size):
    img = segmented_images[i]
    plt.subplot(2, size, i + 1)
    plt.imshow(img)

: 

## Making images bolder to avoid artifacts on resizing

In [ ]:
import cv2
import numpy as np

files = [f for r, d, f in os.walk(SEGMENTED_OUTPUT_DIR)][0]
for f in files:
    filename = SEGMENTED_OUTPUT_DIR + f
    img = cv2.imread(filename, 0)

    kernel = np.ones((2,2), np.uint8)
    dilation = cv2.erode(img, kernel, iterations = 1)
    cv2.imwrite(filename, dilation)

: 

## Converting Segmented Characters to EMNIST format

In [ ]:
def img2emnist(filepath, char_code):
    img = Image.open(filepath).resize((28, 28))
    inv_img = ImageOps.invert(img)
    
    flatten = np.array(inv_img).flatten()
    flatten = flatten / 255
    flatten = np.where(flatten > 0.5, 1, 0)
    
    csv_img = ','.join([str(num) for num in flatten])
    csv_str = '{},{}'.format(char_code, csv_img)
    return csv_str

: 

In [ ]:
temp_filename = 'test.csv'
if os.path.exists(temp_filename):
    os.remove(temp_filename)

f_test = open(temp_filename, 'a+')
column_names = ','.join(["label"] + ["pixel" + str(i) for i in range(784)])
print(column_names, file=f_test)

files = [f for r, d, f in os.walk(SEGMENTED_OUTPUT_DIR)][0]
for f in files:
    csv = img2emnist(SEGMENTED_OUTPUT_DIR + f, -1)
    print(csv, file=f_test)
    
f_test.close()

: 

In [ ]:
test_df = pd.read_csv(temp_filename)
test_df

: 

## Character Recognition

In [ ]:
data = pd.read_csv(temp_filename)
X_data = data.drop(labels = ["label"], axis = 1)
X_data = X_data.values.reshape(-1,28,28,1)

: 

In [ ]:
figure(figsize=(14,14))

size = X_data.shape[0]
for i in range(size):
    v = X_data[i][:,:,0].astype('uint8')
    img = Image.fromarray(255* v)
    
    plt.subplot(2, size, i + 1)
    plt.imshow(img)

: 

In [ ]:
df = pd.read_csv(mapping_processed)
code2char = {}
for index, row in df.iterrows():
    code2char[row['id']] = row['char']

: 

In [ ]:
model = load_model(MODEL_PATH)

: 

In [ ]:
# predict results
results = model.predict(X_data)

# select the index with the maximum probability
results = np.argmax(results, axis = 1)
parsed_str = ""
for r in results:
    parsed_str += code2char[r]

parsed_str

: 

In [ ]:
import re
def build_equation(equa):
    sSplit = []
    operators=[]
    temp=""
    for i in range(len(equa)):
        char=equa[i]
        if char=="*" or char == "/" or char=="+" or char=="-" or char=="=":
          sSplit.append(temp)
          temp=""
          operators.append(char)
        else:
          temp=temp+char
    
    if temp!="":
      sSplit.append(temp)
    # print(operators)
    # print(sSplit)
    main_ans=""
    oper_index=0
    for i in range(len(sSplit)):
        s_temp=sSplit[i]
        i=0;
        first_part=""
        second_part=""
        third_part=""
        while i<len(s_temp) and s_temp[i].isnumeric():
            first_part=first_part+s_temp[i]
            i=i+1
        while i<len(s_temp) and not(s_temp[i].isnumeric()):
            second_part=second_part+s_temp[i]
            i=i+1
        while i<len(s_temp) and s_temp[i].isnumeric():
            third_part=third_part+s_temp[i]
            i=i+1
        ans=""
        if first_part=="":
            if second_part=="":
                ans=third_part
            elif third_part=="":
                ans=second_part
            else:
                ans=second_part+" ** "+third_part
        else:
              if second_part!="":
                if third_part!="":
                    ans=first_part+" * "+second_part+" ** "+third_part+" "
                else:
                    ans = first_part+" * "+second_part+" "
              else:
                ans=first_part
        main_ans=main_ans+ans
        if oper_index<len(operators):
          main_ans=main_ans+" "+ operators[oper_index]+" "
          oper_index=oper_index+1
    print (main_ans)

build_equation(parsed_str)

: 